# EquiLend Data & Analytics | Daily Dispatch Template

## Styling Guidelines

- **Colors**: Use EquiLend's official palette - Blue (primary), Orange, Purple, Grey
- **Font Hierarchy**:
  - Main title (#): Largest font
  - Section titles (##): Second largest
  - Headlines (bold): Third largest
  - Body text: 12pt (smallest)
- **Branding**: Maintain EquiLend's professional, data-driven visual identity
- **Tone**: Professional, use "we/our/us" perspective from EquiLend Data & Analytics
- **Language**: Data-centric assertions ("Our data indicates...", "We observed...")
- **Phrasing**: Use cautious, nuanced language ("may indicate," "could suggest," "appears to be")
- **Citations**: Footnote external sources (WSJ, FT, Bloomberg, Reuters, academic journals)

## Market Notes

- Average Fee (BPS): [Enter Value]
- Average Fee Change (%): [Enter Value]
- Average Utilization (%): [Enter Value]
- Average Utilization Change (%): [Enter Value]

## Sectors in Focus

- [Enter Sector 1]
- [Enter Sector 2]
- [Enter Sector 3]

## Top 5 Headlines

1. **[Headline]**  
   [General color]  
   • **Data:** [color]

2. **[Headline]**  
   [General color]  
   • **Data:** [color]

3. **[Headline]**  
   [General color]  
   • **Data:** [color]

4. **[Headline]**  
   [General color]  
   • **Data:** [color]

5. **[Headline]**  
   [General color]  
   • **Data:** [color]

## Data Tables

### Today's Specials & Hard-to-Borrow (5 Highest with Fee Momentum)
| TICKER | Industry | Fee Range | WoW (%) |
|--------|----------|-----------|---------|
| [Ticker] | [Industry] | [Fee Range] | [WoW %] |

### Global Short Squeeze Score (Top 5)
| Ticker | Industry | Company Description | Short Squeeze Score | Short Interest Indicator (%) |
|--------|----------|-------------------|-------------------|----------------------------|
| [Ticker] | [Industry] | [Company Description] | [SSS Score] | [SI %] |

## Key Takeaways

- [Enter Takeaway 1]
- [Enter Takeaway 2]
- [Enter Takeaway 3]

<a href="https://colab.research.google.com/github/BobSheehan23/EquiLend/blob/main/Enhanced_Gating_%26_Analysis_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Enhanced Gating & Analysis Framework

This notebook implements a sequential gating process to identify securities with the highest potential for short-squeeze risk. The gates are applied in the following order:

1. **Initial Universe**: All securities in the dataset.
2. **On-Loan Value ≥ $1M**: Ensures the security has a meaningful level of short interest.
3. **Borrower Count ≥ 3**: Validates that short demand is broad-based and not driven by a single entity.
4. **Lender Count ≥ 2**: Ensures the supply side is not concentrated, reducing the risk of a single lender's actions creating a misleading signal.
5. **Lendable Inventory > 10% of Float**: Confirms the security is part of the liquid lendable market, avoiding signals driven by artificially small supply.
6. **Utilization ≥ 50%**: Focuses on names where lendable supply is significantly constrained.
7. **Days to Cover**: Used as a tie-breaker for Short Squeeze Score (SSS) ties, not as a pass/fail gate. Higher values indicate it would take multiple days for shorts to exit positions, which can amplify a squeeze.

**Note:**
- *Float* is calculated as: `On Loan Quantity / (Short Interest Indicator / 100)`.
- This process is designed to eliminate common "false positives" and refine the list to only the most compelling candidates.
- Days to Cover serves as a ranking mechanism rather than a filtering criterion.

In [44]:
import pandas as pd
import numpy as np
from docx import Document
from docx.shared import Pt, RGBColor, Inches
from datetime import datetime
import matplotlib.pyplot as plt
import io

# --- Configuration & Setup ---

# Constants for styling the Word document
MAIN_COLOR = RGBColor(0x00, 0x6E, 0xB7)
FONT_NAME = 'Calibri'
DATA_FILE_PATH = 'dd.csv' # Use a descriptive variable
REPORT_DATE = datetime.today()

# --- Helper Functions for Document Generation ---

def add_colored_heading(doc, text, level):
    """Adds a heading with a specific color and font to the document."""
    run = doc.add_heading(level=level).add_run(text)
    run.font.color.rgb = MAIN_COLOR
    run.font.name = FONT_NAME

def add_paragraph(doc, text, size=11, is_bullet=False):
    """Adds a styled paragraph to the document."""
    style = 'List Bullet' if is_bullet else 'Normal'
    p = doc.add_paragraph(text, style=style)
    p.style.font.name = FONT_NAME
    p.style.font.size = Pt(size)
    return p

# --- Core Analysis Functions ---

def load_and_prepare_data(file_path):
    """
    Loads data from the specified CSV and engineers necessary features.
    Handles potential errors during file loading and data processing.
    """
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return None

    # Rename columns for easier access
    df.rename(columns={
        'Utilization (%)': 'Utilization_Pct',
        'On Loan Value (USD)': 'OnLoanValueUSD',
        'Short Interest Indicator': 'ShortInterestPct',
        'Total Lendable Value (USD)': 'LendableValueUSD',
        'Security Price (USD)': 'PriceUSD',
        'Borrower Count': 'BorrowerCount',
        'Lender Count': 'LenderCount',
        'Average Fee': 'AvgFee',
        'On Loan Quantity': 'OnLoanQty',
        'Composite 20-Day ADV': 'ADV20Day'
    }, inplace=True, errors='ignore') # ignore errors if columns don't exist

    # --- Feature Engineering with Error Handling ---
    # Calculate Float as OnLoanQty / (ShortInterestPct / 100)
    # Avoid division by zero if ShortInterestPct is 0 or NaN
    df['Float'] = np.where(df['ShortInterestPct'] > 0, df['OnLoanQty'] / (df['ShortInterestPct'] / 100), 0)

    # Calculate Lendable Shares
    df['LendableShares'] = np.where(df['PriceUSD'] > 0, df['LendableValueUSD'] / df['PriceUSD'], 0)

    # Calculate Lendable as % of Float
    df['LendablePctFloat'] = np.where(df['Float'] > 0, (df['LendableShares'] / df['Float']) * 100, 0)

    # Calculate Days to Cover
    df['DaysToCover'] = np.where(df['ADV20Day'] > 0, df['OnLoanQty'] / df['ADV20Day'], 0)

    # Calculate Market Cap for richer analysis
    df['MarketCapUSD'] = df['Float'] * df['PriceUSD']

    return df

def apply_gating_framework(df):
    """
    Applies a sequence of gates to the DataFrame, tracking the impact of each gate.
    Returns a dictionary of results and a list of filtered-out securities' characteristics.
    """
    if df is None:
        return {}, []

    # Define the gates in the new order:
    gates = {
        '1. Initial Universe': lambda d: d,
        '2. On-Loan Value >= $1M': lambda d: d[d['OnLoanValueUSD'] >= 1_000_000],
        '3. Borrower Count >= 3': lambda d: d[d['BorrowerCount'] >= 3],
        '4. Lender Count >= 2': lambda d: d[d['LenderCount'] >= 2],
        '5. Lendable > 10% of Float': lambda d: d[d['LendablePctFloat'] > 10],
        '6. Utilization >= 50%': lambda d: d[d['Utilization_Pct'] >= 50],
        '7. Days to Cover >= 2': lambda d: d[d['DaysToCover'] >= 2]
    }

    gate_counts = {}
    dropped_securities_analysis = []

    df_filtered = df.copy()
    last_count = len(df_filtered)

    for name, gate_func in gates.items():
        df_after_gate = gate_func(df_filtered)
        current_count = len(df_after_gate)
        gate_counts[name] = current_count

        # Analyze the securities that were dropped by this gate
        if current_count < last_count:
            dropped_indices = df_filtered.index.difference(df_after_gate.index)
            dropped_df = df_filtered.loc[dropped_indices]

            analysis = {
                'Gate': name,
                'Dropped Count': last_count - current_count,
                'Median Market Cap (M)': round(dropped_df['MarketCapUSD'].median() / 1e6, 1),
                'Median Utilization (%)': round(dropped_df['Utilization_Pct'].median(), 1),
                'Median Fee (bps)': round(dropped_df['AvgFee'].median(), 1)
            }
            dropped_securities_analysis.append(analysis)

        df_filtered = df_after_gate
        last_count = current_count

    return gate_counts, dropped_securities_analysis, df_filtered


def generate_impact_chart(gate_counts):
    """Generates a bar chart visualizing the gate attrition and returns it as a bytes object."""
    labels = [label.split('. ')[1] for label in gate_counts.keys()]
    counts = list(gate_counts.values())

    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(labels, counts, color='#006EB7', alpha=0.8)

    ax.set_ylabel('Number of Securities')
    ax.set_title('Gating Framework Attrition', fontsize=16, pad=20)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='x', rotation=45, labelsize=10)
    ax.grid(axis='y', linestyle='--', alpha=0.6)

    # Add count labels on top of bars
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2.0, yval + counts[0]*0.01, f'{yval:,}', ha='center', va='bottom')

    plt.tight_layout()

    # Save plot to a memory buffer
    img_buffer = io.BytesIO()
    plt.savefig(img_buffer, format='png', dpi=300)
    img_buffer.seek(0)
    plt.close(fig) # Close the figure to free memory

    return img_buffer


def create_report_document(gate_counts, dropped_analysis, final_list_df):
    """
    Generates the complete Word document report.
    """
    doc = Document()

    # --- Header ---
    add_colored_heading(doc, 'EquiLend Data & Analytics — Gating Framework Analysis', 0)
    add_paragraph(doc, 'Date: ' + REPORT_DATE.strftime('%d %b %Y'))

    # --- Methodology ---
    add_colored_heading(doc, '1. Sequential Gating Methodology', 1)
    add_paragraph(doc, 'The following sequential filters are applied to identify securities with the highest potential for short-squeeze risk. Each gate is designed to eliminate common "false positives" and refine the list to only the most compelling candidates.')

    # Gate definitions
    gate_rationales = [
        "On-Loan Value ≥ $1M: Ensures the security has a meaningful level of short interest. For example, ABC Corp and XYZ Holdings.",
        "Borrower Count ≥ 3: Validates that short demand is broad-based and not driven by a single entity. For example, DEF Inc and LMN Ltd.",
        "Lender Count ≥ 2: Ensures the supply side is not concentrated, reducing the risk of a single lender's actions creating a misleading signal. For example, QRS Bank and TUV Partners.",
        "Lendable Inventory > 10% of Float: Confirms the security is part of the liquid lendable market, avoiding signals driven by artificially small supply. For example, WXY Group and ZZZ Corp.",
        "Utilization ≥ 50%: Focuses on names where lendable supply is significantly constrained. For example, AAA Tech and BBB Pharma.",
        "Days to Cover ≥ 2: A critical measure indicating it would take multiple days for shorts to exit positions, which can amplify a squeeze. For example, CCC Retail and DDD Energy."
    ]
    for rationale in gate_rationales:
        add_paragraph(doc, rationale, is_bullet=True)

    # --- Impact Analysis ---
    add_colored_heading(doc, '2. Impact on Universe Size', 1)

    # Add summary text
    initial_count = list(gate_counts.values())[0]
    final_count = list(gate_counts.values())[-1]
    reduction_pct = round(100 * (1 - final_count / initial_count), 1) if initial_count > 0 else 0
    add_paragraph(doc, f'The framework reduced an initial universe of {initial_count:,} securities to a final qualified list of {final_count:,} (-{reduction_pct}%). The chart below illustrates the attrition at each stage.')

    # Add the attrition chart
    chart_buffer = generate_impact_chart(gate_counts)
    doc.add_picture(chart_buffer, width=Inches(6.0))

    # --- Analysis of Dropped Securities ---
    add_colored_heading(doc, '3. Profile of Filtered Securities', 1)
    add_paragraph(doc, 'Analyzing the characteristics of securities dropped at each gate provides insight into the function of each filter.')

    if dropped_analysis:
        table = doc.add_table(rows=1, cols=5)
        table.style = 'Table Grid'
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = 'Filtering Gate'
        hdr_cells[1].text = 'Securities Dropped'
        hdr_cells[2].text = 'Median Mkt Cap ($M)'
        hdr_cells[3].text = 'Median Utilization (%)'
        hdr_cells[4].text = 'Median Fee (bps)'

        for item in dropped_analysis:
            row_cells = table.add_row().cells
            row_cells[0].text = item['Gate'].split('. ')[1]
            row_cells[1].text = f"{item['Dropped Count']:,}"
            row_cells[2].text = str(item['Median Market Cap (M)'])
            row_cells[3].text = str(item['Median Utilization (%)'])
            row_cells[4].text = str(item['Median Fee (bps)'])

    # --- Final Candidate List ---
    add_colored_heading(doc, '4. Final Candidate List', 1)
    add_paragraph(doc, f"The following {len(final_list_df)} securities passed all gates.")
    if not final_list_df.empty:
        # Add a table of the top candidates
        display_cols = ['Ticker', 'OnLoanValueUSD', 'Utilization_Pct', 'DaysToCover', 'AvgFee', 'BorrowerCount']
        final_list_display = final_list_df[display_cols].sort_values(by='DaysToCover', ascending=False).head(10)

        # Format for display
        final_list_display['OnLoanValueUSD'] = final_list_display['OnLoanValueUSD'].apply(lambda x: f"${x/1e6:,.1f}M")
        final_list_display['Utilization_Pct'] = final_list_display['Utilization_Pct'].apply(lambda x: f"{x:.1f}%")
        final_list_display['DaysToCover'] = final_list_display['DaysToCover'].apply(lambda x: f"{x:.2f}")

        table = doc.add_table(rows=1, cols=len(display_cols))
        table.style = 'Table Grid'
        hdr_cells = table.rows[0].cells
        for i, col_name in enumerate(final_list_display.columns):
            hdr_cells[i].text = col_name

        for index, row in final_list_display.iterrows():
            row_cells = table.add_row().cells
            for i, val in enumerate(row):
                row_cells[i].text = str(val)


    # --- Save the document ---
    file_name = f'EquiLend_Gating_Analysis_{REPORT_DATE.strftime("%Y-%m-%d")}.docx'
    doc.save(file_name)
    print(f"Successfully generated report: '{file_name}'")


# --- Main Execution ---
if __name__ == "__main__":
    # 1. Load and prepare the data
    raw_df = load_and_prepare_data(DATA_FILE_PATH)

    if raw_df is not None:
        # 2. Apply the gating framework
        gate_results, dropped_stats, final_df = apply_gating_framework(raw_df)

        # 3. Generate the final Word document report
        create_report_document(gate_results, dropped_stats, final_df)

TypeError: '>' not supported between instances of 'str' and 'int'

In [29]:
# --- Daily Dispatch Word Document Generator ---

def create_daily_dispatch_document():
    """
    Creates a formatted Daily Dispatch Word document following EquiLend's branding guidelines.
    """
    doc = Document()
    
    # Set up EquiLend colors
    EQUILEND_BLUE = RGBColor(0x00, 0x6E, 0xB7)
    EQUILEND_ORANGE = RGBColor(0xFF, 0x8C, 0x00)
    EQUILEND_PURPLE = RGBColor(0x6B, 0x46, 0xC1)
    EQUILEND_GREY = RGBColor(0x6B, 0x73, 0x80)
    
    # --- Main Title (Largest font) ---
    title = doc.add_heading(level=0)
    title_run = title.add_run('EquiLend Data & Analytics | Daily Dispatch')
    title_run.font.color.rgb = EQUILEND_BLUE
    title_run.font.name = FONT_NAME
    title_run.font.size = Pt(24)
    
    # Add date
    date_para = doc.add_paragraph()
    date_run = date_para.add_run(f'Date: {REPORT_DATE.strftime("%B %d, %Y")}')
    date_run.font.name = FONT_NAME
    date_run.font.size = Pt(12)
    
    # --- Market Notes Section ---
    market_heading = doc.add_heading(level=1)
    market_run = market_heading.add_run('Market Notes')
    market_run.font.color.rgb = EQUILEND_BLUE
    market_run.font.name = FONT_NAME
    market_run.font.size = Pt(18)
    
    market_notes = [
        "Average Fee (BPS): [Enter Value]",
        "Average Fee Change (%): [Enter Value]", 
        "Average Utilization (%): [Enter Value]",
        "Average Utilization Change (%): [Enter Value]"
    ]
    
    for note in market_notes:
        para = doc.add_paragraph()
        run = para.add_run(f"• {note}")
        run.font.name = FONT_NAME
        run.font.size = Pt(12)
    
    # --- Sectors in Focus ---
    sectors_heading = doc.add_heading(level=1)
    sectors_run = sectors_heading.add_run('Sectors in Focus')
    sectors_run.font.color.rgb = EQUILEND_BLUE
    sectors_run.font.name = FONT_NAME
    sectors_run.font.size = Pt(18)
    
    sectors = ["[Enter Sector 1]", "[Enter Sector 2]", "[Enter Sector 3]"]
    for sector in sectors:
        para = doc.add_paragraph()
        run = para.add_run(f"• {sector}")
        run.font.name = FONT_NAME
        run.font.size = Pt(12)
    
    # --- Top 5 Headlines ---
    headlines_heading = doc.add_heading(level=1)
    headlines_run = headlines_heading.add_run('Top 5 Headlines')
    headlines_run.font.color.rgb = EQUILEND_BLUE
    headlines_run.font.name = FONT_NAME
    headlines_run.font.size = Pt(18)
    
    for i in range(1, 6):
        # Headline number and title
        headline_para = doc.add_paragraph()
        headline_num = headline_para.add_run(f"{i}. ")
        headline_num.font.name = FONT_NAME
        headline_num.font.size = Pt(14)
        headline_num.font.bold = True
        
        headline_text = headline_para.add_run("[Headline]")
        headline_text.font.name = FONT_NAME
        headline_text.font.size = Pt(14)
        headline_text.font.bold = True
        headline_text.font.color.rgb = EQUILEND_BLUE
        
        # General color line
        color_para = doc.add_paragraph()
        color_run = color_para.add_run("   [General color]")
        color_run.font.name = FONT_NAME
        color_run.font.size = Pt(12)
        color_run.font.italic = True
        
        # Data line
        data_para = doc.add_paragraph()
        data_bullet = data_para.add_run("   • ")
        data_bullet.font.name = FONT_NAME
        data_bullet.font.size = Pt(12)
        
        data_label = data_para.add_run("Data: ")
        data_label.font.name = FONT_NAME
        data_label.font.size = Pt(12)
        data_label.font.bold = True
        data_label.font.color.rgb = EQUILEND_ORANGE
        
        data_content = data_para.add_run("[color]")
        data_content.font.name = FONT_NAME
        data_content.font.size = Pt(12)
        
        # Add spacing between headlines
        doc.add_paragraph()
    
    # --- Data Tables ---
    tables_heading = doc.add_heading(level=1)
    tables_run = tables_heading.add_run('Data Tables')
    tables_run.font.color.rgb = EQUILEND_BLUE
    tables_run.font.name = FONT_NAME
    tables_run.font.size = Pt(18)
    
    # --- Today's Specials & Hard-to-Borrow Table ---
    htb_subheading = doc.add_paragraph()
    htb_icon = htb_subheading.add_run("📊 ")
    htb_icon.font.size = Pt(14)
    htb_text = htb_subheading.add_run("Today's Specials & Hard-to-Borrow (5 Highest with Fee Momentum)")
    htb_text.font.name = FONT_NAME
    htb_text.font.size = Pt(14)
    htb_text.font.bold = True
    htb_text.font.color.rgb = EQUILEND_BLUE
    
    # Create HTB table
    htb_table = doc.add_table(rows=6, cols=4)
    htb_table.style = 'Table Grid'
    
    # Header row
    htb_headers = ['TICKER', 'Industry', 'Fee Range', 'WoW (%)']
    for i, header in enumerate(htb_headers):
        cell = htb_table.rows[0].cells[i]
        cell.text = header
        for paragraph in cell.paragraphs:
            for run in paragraph.runs:
                run.font.bold = True
                run.font.name = FONT_NAME
                run.font.size = Pt(11)
    
    # Sample data rows
    htb_data = [
        ['XTIA', 'Software and Services', '50,000+', '65.00'],
        ['ACXP', 'Biotechnology', '30,000 to 49,999', '137,000.00'],
        ['OST', 'Technology Hardware and Equipment', '30,000 to 49,999', '60.00'],
        ['NEHC', 'Oil and Gas Exploration and Production', '10,000 to 29,999', '188.00'],
        ['PCSA', 'Pharmaceuticals', '10,000 to 29,999', '871.00']
    ]
    
    for i, row_data in enumerate(htb_data, 1):
        for j, cell_data in enumerate(row_data):
            cell = htb_table.rows[i].cells[j]
            cell.text = cell_data
            for paragraph in cell.paragraphs:
                for run in paragraph.runs:
                    run.font.name = FONT_NAME
                    run.font.size = Pt(10)
    
    doc.add_paragraph()  # Add spacing
    
    # --- Global Short Squeeze Score Table ---
    sss_table = doc.add_table(rows=6, cols=5)
    sss_table.style = 'Table Grid'
    
    # Header row
    sss_headers = ['Ticker', 'Industry', 'Company Description', 'Short Squeeze Score', 'Short Interest Indicator (%)']
    for i, header in enumerate(sss_headers):
        cell = sss_table.rows[0].cells[i]
        cell.text = header
        for paragraph in cell.paragraphs:
            for run in paragraph.runs:
                run.font.bold = True
                run.font.name = FONT_NAME
                run.font.size = Pt(10)
    
    # Sample data rows
    sss_data = [
        ['5721 JP', 'Materials', 'Steel processing and manufacturing', '84.0', '9.69'],
        ['3350 JP', 'Consumer Discretionary', 'Retail and lifestyle products', '73.0', '20.57'],
        ['032820 KS', 'Information Technology', 'Software development and services', '73.0', '1.52'],
        ['ALLMG FP', 'Information Technology', 'Digital solutions and consulting', '72.0', '3.33'],
        ['4584 JP', 'Health Care', 'Pharmaceutical research and development', '70.0', '29.95']
    ]
    
    for i, row_data in enumerate(sss_data, 1):
        for j, cell_data in enumerate(row_data):
            cell = sss_table.rows[i].cells[j]
            cell.text = cell_data
            for paragraph in cell.paragraphs:
                for run in paragraph.runs:
                    run.font.name = FONT_NAME
                    run.font.size = Pt(9)
    
    # --- Key Takeaways ---
    takeaways_heading = doc.add_heading(level=1)
    takeaways_run = takeaways_heading.add_run('Key Takeaways')
    takeaways_run.font.color.rgb = EQUILEND_BLUE
    takeaways_run.font.name = FONT_NAME
    takeaways_run.font.size = Pt(18)
    
    takeaways = ["[Enter Takeaway 1]", "[Enter Takeaway 2]", "[Enter Takeaway 3]"]
    for takeaway in takeaways:
        para = doc.add_paragraph()
        run = para.add_run(f"• {takeaway}")
        run.font.name = FONT_NAME
        run.font.size = Pt(12)
    
    # --- Footer with Styling Guidelines ---
    doc.add_page_break()
    
    guidelines_heading = doc.add_heading(level=1)
    guidelines_run = guidelines_heading.add_run('Styling Guidelines Reference')
    guidelines_run.font.color.rgb = EQUILEND_GREY
    guidelines_run.font.name = FONT_NAME
    guidelines_run.font.size = Pt(16)
    
    guidelines_text = [
        "Colors: EquiLend's official palette - Blue (primary), Orange, Purple, Grey",
        "Font Hierarchy: Main title (largest) → Section titles → Headlines → Body text (12pt)",
        "Tone: Professional, use 'we/our/us' perspective from EquiLend Data & Analytics",
        "Language: Data-centric assertions ('Our data indicates...', 'We observed...')",
        "Phrasing: Use cautious, nuanced language ('may indicate,' 'could suggest,' 'appears to be')",
        "Citations: Footnote external sources (WSJ, FT, Bloomberg, Reuters, academic journals)"
    ]
    
    for guideline in guidelines_text:
        para = doc.add_paragraph()
        run = para.add_run(f"• {guideline}")
        run.font.name = FONT_NAME
        run.font.size = Pt(10)
        run.font.color.rgb = EQUILEND_GREY
    
    # Save the document
    dispatch_filename = f'EquiLend_Daily_Dispatch_{REPORT_DATE.strftime("%Y-%m-%d")}.docx'
    doc.save(dispatch_filename)
    print(f"Successfully generated Daily Dispatch document: '{dispatch_filename}'")
    
    return dispatch_filename

# Generate the Daily Dispatch Word document
if __name__ == "__main__":
    dispatch_file = create_daily_dispatch_document()
    print(f"Daily Dispatch document created: {dispatch_file}")

Successfully generated Daily Dispatch document: 'EquiLend_Daily_Dispatch_2025-06-23.docx'
Daily Dispatch document created: EquiLend_Daily_Dispatch_2025-06-23.docx


In [30]:
def create_daily_dispatch_html(date_str, market_notes_html, sectors_html, headlines_html, htb_table, sss_table, takeaways_html):
    html_content = f'''
<!DOCTYPE html>
<html>
<head><title>EquiLend D&A Daily Digest</title></head>
<body style="font-family:Calibri,Arial,sans-serif;margin:20px;padding:0;background-color:#f4f4f4;color:#333333;">
<table align="center" style="width:800px;border:0;cellspacing:0;cellpadding:0;background-color:#ffffff;margin:20px auto;font-size:12px;">
<tr>
<td>
<h1 style="font-size:26px;color:#007ab8;margin:0;font-weight:bold;">EquiLend D&A Daily Digest</h1>
<div style="font-size:12px;color:#FF8C00;font-weight:bold;">{date_str}</div>
 
    <h2 style="font-size:16px;color:#007ab8;margin-top:20px;margin-bottom:10px;font-weight:bold;">📈 MARKET NOTES:</h2>
    <div>{market_notes_html}</div>
 
    <h2 style="font-size:16px;color:#007ab8;margin-top:20px;margin-bottom:10px;font-weight:bold;">📊 Sectors to Watch:</h2>
    <ul style="padding-left:20px;">{sectors_html}</ul>
 
    <h2 style="font-size:16px;color:#007ab8;margin-top:20px;margin-bottom:10px;font-weight:bold;">🔥 Major Headlines & What Our Data Shows</h2>
    {headlines_html}
 
    <h2 style="font-size:16px;color:#007ab8;margin-top:20px;margin-bottom:10px;font-weight:bold;">📊 Today's Specials & Hard-to-Borrow (5 Highest with Fee Momentum)</h2>
    {htb_table}
 
    <h2 style="font-size:16px;color:#007ab8;margin-top:20px;margin-bottom:10px;font-weight:bold;">📈 Global Top 5 Short Squeeze Scores</h2>
    {sss_table}
 
    <h2 style="font-size:16px;color:#007ab8;margin-top:20px;margin-bottom:10px;font-weight:bold;">💡 Key Takeaways</h2>
    <ul style="padding-left:20px;">{takeaways_html}</ul>
 
</td>
</tr>
</table>
</body>
</html>
    '''
    return html_content


In [31]:
# Combine equity and fixed income CSVs into dd.csv for analysis
import pandas as pd
from pathlib import Path

# Paths to source files
csv_eq = Path("/Users/bob/Library/Mobile Documents/com~apple~CloudDocs/OneDrive_1_6-23-2025/ddeq_2025-06-23_bsheehan_adhoc.csv")
csv_fi = Path("/Users/bob/Library/Mobile Documents/com~apple~CloudDocs/OneDrive_1_6-23-2025/DDFI_2025-06-23_bsheehan_adhoc.csv")
combined_csv = Path("/Users/bob/Library/Mobile Documents/com~apple~CloudDocs/Work_Github/dd.csv")

# Read and concatenate
if csv_eq.exists() and csv_fi.exists():
    df_eq = pd.read_csv(csv_eq, header=None)
    df_fi = pd.read_csv(csv_fi, header=None)
    df_combined = pd.concat([df_eq, df_fi], ignore_index=True)
    df_combined.to_csv(combined_csv, index=False, header=False)
    print(f"Combined CSV written to: {combined_csv}")
else:
    print("One or both source files not found.")

Combined CSV written to: /Users/bob/Library/Mobile Documents/com~apple~CloudDocs/Work_Github/dd.csv


In [ ]:
# --- Write Top 5 Headlines with Static Data Bullet Placeholder ---
from pathlib import Path
from IPython.display import Markdown, display

headlines_md_path = f"/Users/bob/Desktop/daily_dispatch_headlines_{REPORT_DATE.strftime('%Y-%m-%d')}.md"

headline_narratives = [
    "Geopolitics: Iran–Israel Conflict Escalates; US Strikes Involved\n   US military bombed major Iranian nuclear sites (Fordow, Natanz, Isfahan) over the weekend. Iran responded with missile and drone attacks on Israel and threatened “everlasting consequences.” Oil prices surged as the Strait of Hormuz faces new risks.",
    "Monetary Policy: Fed Signals Measured Rate Cuts Amid Persistent Inflation\n   The Federal Reserve maintains a cautious stance on future rate cuts as inflation cools but remains above target.",
    "Financial Markets: Global Equities Rally, MSCI World Hits New High\n   Global equities extend gains, led by US large-cap tech; MSCI World Index reaches a fresh high.",
    "US–China Relations: Semiconductor Policy at Center of Renewed Talks\n   US and China trade representatives reopen talks in Washington, focusing on semiconductor export controls.",
    "Energy Policy: OPEC⁺ Maintains Production Curbs, Oil Stabilizes\n   OPEC⁺ keeps existing production curbs in place; crude stabilizes near $80/bbl, easing near-term supply concerns."
]

headlines_md = "# Top 5 Headlines\n\n"
for i, headline in enumerate(headline_narratives, 1):
    headlines_md += f"{i}. **{headline}**  \n   • **Data:** *insert relevant data point from related securities with elevated metrics or large deltas*\n\n"

with open(headlines_md_path, "w", encoding="utf-8") as f:
    f.write(headlines_md)

display(Markdown(headlines_md))


1. Geopolitics: Iran–Israel Conflict Escalates; US Strikes Involved
   US military bombed major Iranian nuclear sites (Fordow, Natanz, Isfahan) over the weekend. Iran responded with missile and drone attacks on Israel and threatened “everlasting consequences.” Oil prices surged as the Strait of Hormuz faces new risks.

   - Data: *insert relevant data point from related securities with elevated metrics or large deltas*

2. Monetary Policy: Fed Signals Measured Rate Cuts Amid Persistent Inflation
   The Federal Reserve maintains a cautious stance on future rate cuts as inflation cools but remains above target.

   - Data: *insert relevant data point from related securities with elevated metrics or large deltas*

3. Financial Markets: Global Equities Rally, MSCI World Hits New High
   Global equities extend gains, led by US large-cap tech; MSCI World Index reaches a fresh high.

   - Data: *insert relevant data point from related securities with elevated metrics or large deltas*

4. US–China Relations: Semiconductor Policy at Center of Renewed Talks
   US and China trade representatives reopen talks in Washington, focusing on semiconductor export controls.

   - Data: *insert relevant data point from related securities with elevated metrics or large deltas*

5. Energy Policy: OPEC⁺ Maintains Production Curbs, Oil Stabilizes
   OPEC⁺ keeps existing production curbs in place; crude stabilizes near $80/bbl, easing near-term supply concerns.

   - Data: *insert relevant data point from related securities with elevated metrics or large deltas*

